<a href="https://colab.research.google.com/github/hankedwards/AppleWatch/blob/master/InvestmentTable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generate rate of return table for each investment in Investdb3


In [ ]:
import datetime as dt
# import pytz
import sqlite3
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import datetime, date, timedelta

## Create dataframe from SQLite database

### Set up colab to access google drive

In [ ]:
from google.colab import drive
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

## Connect to the database

In [ ]:
 # # def readdata():
db_conn = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/Net Worth Experiments/invest3.db')
df = pd.read_sql_query("SELECT * from Inv2", db_conn)
# Covert to date object

In [ ]:
print(df.info())

### Set up Colab data frame formatter

In [ ]:
from google.colab import data_table
# from vega_datasets import data

data_table.enable_dataframe_formatter()

## Display the table using Colab table formatter

In [ ]:
df

In [ ]:
df['dtdate'] = pd.to_datetime(df['Date'], format='%Y-%m-%d').dt.strftime('%m/%d/%Y')
df['dtmonth'] =pd.DatetimeIndex(df['dtdate']).month
df['dtyear'] =pd.DatetimeIndex(df['dtdate']).year

In [ ]:
yeardf = df[(df['dtyear'] >= 2017) ]
yeardf.sort_values(by=['Investment', 'Date'], ascending =[True,True ],inplace = True)
yeardf

In [ ]:
yeardf.reindex()

In [ ]:
uniinv = yeardf.Investment.unique()
uniinv =['BL Nationwide', 'Cheryl Fidelity','Novartis/Fidelity', 'WellsFargo Henry', 'WellsFargo Cheryl']
uniinv.sort()
uniinv                

In [ ]:
yearrange = range(2017, 2023)

##Loop through investments and years to create list 


In [ ]:
firsttime = True
invlist =[]
for inv in uniinv:
  for yr in yearrange:
    invdf = yeardf.loc[(yeardf['Investment'] == inv) & (yeardf['dtyear']== yr)].head(1)
    invdffull = yeardf.loc[(yeardf['Investment'] == inv) & (yeardf['dtyear']== yr)]
    begamt = invdf["Amount"].values[0]
    begmt = "${:,}".format(begamt)
    begdate = invdf["dtdate"].values[0]
    maxamt = "${:,}".format(invdffull["Amount"].max())
    minamt = "${:,}".format(invdffull["Amount"].min())
    avgamt = "${:,.0f}".format(invdffull["Amount"].mean())
    taildf = yeardf[(yeardf['Investment'] == inv) & (yeardf['dtyear']== yr)].tail(1)  
    endamt = taildf["Amount"].values[0]
    endamt1 = "${:,}".format(endamt)
    enddate = taildf["dtdate"].values[0] 
    ratereturn = ((endamt - begamt)/begamt)*100
    rtereturn = (round(ratereturn,2))
    # rtereturn = "{:6.4f}".format(ratereturn)
    invlist = [inv, yr, maxamt, minamt, avgamt, begmt, endamt1, rtereturn]
    if firsttime:
      invlist2 = [invlist]
      firsttime = False
    else:
      invlist2.append(invlist)


In [ ]:
print(invlist2)

In [ ]:
dfudf = DataFrame(invlist2, columns=['inv', 'yr', 'maxamt', 'minamt', 'avgamt', 'begmt', 'endamt1', 'rtereturn'])

In [ ]:
dfudf

In [ ]:
# dfudf.sort_values(by=['yr','inv'])
import plotly.express as px

fig = px.bar(dfudf, x="yr", y="rtereturn",color="inv", barmode = 'group',
      text_auto=True)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.85
))

fig.update_layout(title_text='<b>Rate of Return by Investment', title_x=0.5) 

fig.show()

In [ ]:
spdf = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Net Worth Experiments/SP historical.xlsx') 
spdf['rtereturn'] = (round(spdf['rtereturn'],2))
spdf

In [ ]:
spdf.info()

dfudf.info()
combined = pd.concat([spdf, dfudf])
combined.info()

In [ ]:
fig = px.bar(combined, x="yr", y="rtereturn",color="inv", barmode = 'group',
      text="rtereturn")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.85
))

fig.update_layout(title_text='<b>Rate of Return by Investment', title_x=0.5)
                  

fig.show()

In [ ]:
!pip install yfinance

In [ ]:
import yfinance
# tsla = yfinance.Ticker('FBALX') FMSDX
fmsdx = yfinance.Ticker('FMSDX')
fmsdx1 = fmsdx.history(period='2y')
fbalx = yfinance.Ticker('FBALX')
fbalx1 = fbalx.history(period='2y')
spxx = yfinance.Ticker('^GSPC')
spxx1 = spxx.history(period='2y')
comp = yfinance.Ticker('COMP')
comp1 = comp.history(period='2y')

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=spxx1.index,y=spxx1['Close'], mode='lines'))
fig.update_layout(title_text='<b>S&P 500</b>', title_x=0.5)
# fig = go.Figure(data=go.Scatter(x=fmsdx1.index,y=fmsdx1['Close'], mode='lines'))
fig.show()

In [ ]:
# import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
# fig = go.Figure(data=go.Scatter(x=fmsdx1.index,y=hist['Close'], mode='lines'))
# fig.add_trace(
#     go.Scatter(x=comp1.index,y=comp1['Close'], mode='lines'),
#     secondary_y=False,
# )
# fig.add_trace(
#     go.Scatter(x=[2, 3, 4], y=[4, 5, 6], name="yaxis2 data"),
#     secondary_y=True,
fig.add_trace(
    go.Scatter(x=spxx1.index,y=spxx1['Close'], mode='lines'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=fbalx1.index,y=fbalx1['Close'], mode='lines'),
    secondary_y=True,
)
# Add figure title
fig.update_layout(
    title_text="<b>Stock Index Comparison</b>"
)

# Set x-axis title
fig.update_xaxes(title_text="Time Frame")

# Set y-axes titles
fig.update_yaxes(title_text="<b>S&P 500</b> ", secondary_y=False)
fig.update_yaxes(title_text="<b>FBALX</b> ", secondary_y=True)

fig.show()

## Disabling Data Table Display

To restore the native pandas dataframe display, use the following:

In [ ]:
# from google.colab import data_table
# data_table.disable_dataframe_formatter()

# data.airports()